# Compute Spectra Workflow
- Identify material system of interest
  - optionally choose from Materials Project
- Specify simulation parameters
- Run simulation
- Extract simulation outputs

In [ ]:
%%html
<script>
    // AUTORUN APPMODE ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function (location) {
                location.replace(window.location.href.replace("notebooks", "apps"));
            });
        }
    );
</script>

###  Import Dependencies

In [1]:
from hublib.ui import Tab, Form
from frontend import *

<IPython.core.display.Javascript object>

SyntaxError: invalid syntax (frontend.py, line 94)

## Materials Project Authentication (Optional):

In [ ]:
Authenticate()

## Pick a Semiconductor to Simulate

In [ ]:
q = Query(authfile="~/.mpkey.txt")

In [ ]:
structure_box = Tab([q.qgrid()], titles=["Online"])

In [ ]:

        
    
output = widgets.Output()
mpid.observe(update_plot,'value')  

display(mpid, plot_button, output)

# filter by pretty_formula to look for a specific structure
display(qgrid)

### Get structure from MP database
TODO: could be combined with the last cell

In [ ]:
struct = rester.get_structure_by_material_id("mp-"+str(mpid.value), final = False, conventional_unit_cell=True)
struct_dict = struct.as_dict()#to(fmt="JSON")
struct_dict

### Choice (b): upload your own poscar directly. No query necessary.

In [ ]:
#users can pass their own structure as a poscar. the simtool will take the file contents as a dictionary which is produced here
struct = Structure.from_file("./POSCAR")
struct_dict = struct.as_dict()
struct_dict

# Perform structure relaxation, scf, and phonon computation and spectra extraction using simtool

### Find relax_sim simtool notebook and confirm

In [ ]:
#simToolName = "670raman"
simToolName = "relax_sim"
simToolLocation = searchForSimTool(simToolName)
for key in simToolLocation.keys():
    print(f"{key} = {simToolLocation[key]}")

In [ ]:
# get list of available pseudopotential files

pp_list = []

for filename in os.listdir("./simtool/pseudo/"):
    f = os.path.join("./simtool/pseudo/", filename)
    # get a list of all the PPs -- is this best instatiated here or globally?
    # if instanced here, the user could probably pass their own PPs to the constructor as well
    if os.path.isfile(f):
        pp_list.append(filename)

# filter by selected compound compositions
elements = np.unique([''.join([i for i in str(i.species) if i.isalpha()]) for i in struct.sites])
filtered_pp_list = [pp for pp in pp_list for e in elements if e in pp]

# make widgets for sim2l parameters

log = widgets.Select(
    options=["DEBUG", "INFO", "WARNING", "ERROR", "CRITICAL"],
    value='DEBUG',
    # rows=10,
    description='Log Level:',
)

walltime = widgets.Text(
    value='01:00:00',
    placeholder='walltime',
    description='walltime:',
    disabled=False
)
numnodes = widgets.IntText(
    value=8,
    placeholder='nodes',
    description='nodes:',
    disabled=False
) 
button = widgets.Button(
    description='run simtool',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='run to submit qe simtool'
)
ecutwfc = widgets.BoundedFloatText(
    value=50,
    min=50,
    max=400,
    step=10,
    description='ecutwfc:',
    disabled=False
)
kpoints = widgets.BoundedFloatText(
    value=6,
    min=1,
    max=20,
    step=1,
    description='kpoints:',
    disabled=False
)
ecutrho = widgets.BoundedFloatText(
    value=200,
    min=200,
    max=1600,
    step=40,
    description='ecutrho:',
    disabled=False
)
smearing = widgets.Select(
    options=['smearing','fixed'],
    value='fixed',
    rows = 2,
    description='smearing:',
    disabled=False
)
pp_menu1 = widgets.Combobox(
    placeholder="choose a pseudopotential",
    options=filtered_pp_list,
    description='pseudo 1:',
    disabled=False
)
pp_menu2 = widgets.Combobox(
    placeholder="choose a pseudopotential",
    options=filtered_pp_list,
    description='pseudo 2:',
    disabled=False
)

output = widgets.Output()

def runSim2l():
    inputs['loglevel'].value = log.value
    inputs['walltime'].value = walltime.value
    inputs['numnodes'].value = numnodes.value
    inputs['ecutwfc'].value = ecutwfc.value
    inputs['ecutrho'].value = ecutrho.value
    inputs['kpoints'].value = kpoints.value
    inputs['pps'].value = [pp_menu1.value, pp_menu2.value]
    inputs['smearing'].value = smearing.value
    inputs['struct_dict'].value = struct_dict

def on_button_clicked(b):
    with output:
        runSim2l()
        r = Run(simToolLocation,inputs)
        results = r.getResultSummary()
        print(r.read('spectra'))
    return results
        
        
results = button.on_click(on_button_clicked)

simulation = widgets.VBox([ecutrho, ecutwfc, kpoints, smearing, pp_menu1, pp_menu2])
run_details = widgets.VBox([walltime, numnodes, log])

accordion = widgets.VBox([widgets.Accordion(children=[simulation,run_details]),button,output])
display(accordion)

### Or, iterate through inputs directly

### Simulator Outputs

In [ ]:
outputs = getSimToolOutputs(simToolLocation)
outputs

### Run simtool to obtain Predicted Raman Tensor and Spectrum Graph

In [ ]:
r.getResultSummary()

In [ ]:
results.read('spectra')

In [ ]:
r.read('spectra')

In [ ]:
print(r.read('logreport'))

In [ ]:
#check inputs
r.input_dict

In [ ]:
#find output location
print(r.outdir)